In [1]:
import torch 
import matplotlib.pyplot as plt
from utils import * 
from mlmm import SmoothKernelMLMM
from einops import rearrange, repeat
import numpy as np

In [2]:
torch.set_printoptions(precision=2, sci_mode=False, linewidth=100)

In [3]:
# singular smooth kernel 
l = 5
n = 2**l - 1
xmin = -1
xmax = 1
h = (xmax - xmin)/(n+1)

xh = torch.linspace(-1, 1, n+2)[1:-1][None][None] 

# input function
uh = 1-xh**2

# kernel function
gh_X, gh_Y = torch.meshgrid(xh[0,0], xh[0,0])
Khh = torch.log((gh_Y - gh_X).abs()+1e-5)[None][None]

# output function(numeric)
wh = multi_summation(Khh, uh, h)

/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


MLMM singular smooth case

In [4]:
k = 2

hs = []
Ks = []
us = []

for i in range(k):
    if i == 0:
        hs.append(h)
        Ks.append(Khh)
        us.append(uh)
        
        H = h*2
        KHH = Khh[:,:,1::2,1::2]
        uH = restrict1d(uh)

    else:
        H = H*2
        KHH = KHH[:,:,1::2,1::2]
        uH = restrict1d(uH)
    
    hs.append(H)
    Ks.append(KHH)
    us.append(uH)

In [5]:
# two grid fast matrix multiplcation
# fine grid, level 1
KHH = Ks[-1]
uH = us[-1]
H = hs[-1]

Khh = Ks[-2]
uh = us[-2]
h = hs[-2]
n = uh.shape[-1]

In [7]:
# fetch nbrs idx
m = 2
idx_i, idx_j = fetch_nbrs(n, m)

In [11]:
idx_i_even = idx_i[1::2]
idx_j_even = idx_j[1::2]

In [15]:
KHH.shape

torch.Size([1, 1, 15, 15])

In [14]:
idx_j_even

tensor([[ 0,  1,  2,  3,  4],
        [ 2,  3,  4,  5,  6],
        [ 4,  5,  6,  7,  8],
        [ 6,  7,  8,  9, 10],
        [ 8,  9, 10, 11, 12],
        [10, 11, 12, 13, 14],
        [12, 13, 14, 15, 16],
        [14, 15, 16, 17, 18],
        [16, 17, 18, 19, 20],
        [18, 19, 20, 21, 22],
        [20, 21, 22, 23, 24],
        [22, 23, 24, 25, 26],
        [24, 25, 26, 27, 28],
        [26, 27, 28, 29, 30],
        [28, 29, 30, 31, 32]])

In [12]:
idx_i_even

tensor([[ 2,  2,  2,  2,  2],
        [ 4,  4,  4,  4,  4],
        [ 6,  6,  6,  6,  6],
        [ 8,  8,  8,  8,  8],
        [10, 10, 10, 10, 10],
        [12, 12, 12, 12, 12],
        [14, 14, 14, 14, 14],
        [16, 16, 16, 16, 16],
        [18, 18, 18, 18, 18],
        [20, 20, 20, 20, 20],
        [22, 22, 22, 22, 22],
        [24, 24, 24, 24, 24],
        [26, 26, 26, 26, 26],
        [28, 28, 28, 28, 28],
        [30, 30, 30, 30, 30]])